# tmp
this is just a dump for testing

In [1]:
import os
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
sys.path.insert(0, '../../icenumerics/')
sys.path.insert(0, './auxnumerics/')
sys.path.insert(0, './testing/')
import icenumerics as ice

from parameters import params
import auxiliary as aux
import montecarlo_colloids as mc
import vertices as vrt

ureg = ice.ureg

%reload_ext autoreload
%autoreload 2

idx = pd.IndexSlice

In [2]:
plt.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

In [3]:
N = 10
a = params['lattice_constant']

sp = ice.spins()
sp.create_lattice('square', [N,N], lattice_constant=a ,border='periodic')

particle = ice.particle(radius=params['particle_radius'],
                        susceptibility=params['particle_susceptibility'],
                        diffusion=params['particle_diffusion'],
                        temperature=params['particle_temperature'],
                        density=params['particle_density']
)

trap = ice.trap(trap_sep=params['trap_sep'],
                height=params['trap_height'],
                stiffness=params['trap_stiffness']
)

trap2 = ice.trap(trap_sep=0*params['trap_sep'],
                height=params['trap_height'],
                stiffness=params['trap_stiffness']
)


GSTRJ = pd.read_csv(f'../data/states/ice/{N}.csv',index_col='id')

# make the structure of the colloid
centers = [row[['x','y','z']].to_list() * ureg.um for _,row in GSTRJ.iterrows()]
directions = [row[['dx','dy','dz']].to_list() * ureg.um for _,row in GSTRJ.iterrows()]
arrangement = {"centers": centers,
              "directions": directions
              }

# select one particle to fix
idx = 42
traps = [trap2 if (i==idx) else trap for i in range(len(centers)) ]

col = ice.colloidal_ice(arrangement, particle, traps,
                       height_spread=0,
                       susceptibility_spread=0.1,
                       periodic=True)

particle_radius = params['particle_radius']
col.region = np.array([[0,0,-3*(particle_radius/a/N).magnitude],[1,1,3*(particle_radius/a/N).magnitude]])*N*a

# move the particle a tiny bit 
col[idx].colloid += col[idx].direction * params['lattice_constant']/8
col[idx].center += col[idx].direction * params['trap_sep']/2

col.display()


FileNotFoundError: [Errno 2] No such file or directory: '../data/states/ice/10.csv'

In [ ]:
col[idx].colloid

In [ ]:
col.to_ctrj()

In [ ]:
col[idx].colloid += col[idx].direction * params['lattice_constant']/4


In [ ]:
col[idx].colloid 

In [ ]:

col = ice.colloidal_ice(sp, particle, trap,
                        height_spread=params['height_spread'],
                        susceptibility_spread=params['susceptibility_spread'],
                        periodic=params['isperiodic']
)

col.randomize()
col.region = np.array([[0,0,-3*(params["particle_radius"]/a/N).magnitude],[1,1,3*(params["particle_radius"]/a/N).magnitude]])*N*a

col.to_ctrj().to_csv('array.csv')

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))

ice.draw_frame(trj, frame_no=0,
               region=[0,300,0,300],
               radius=params["particle_radius"].magnitude,
               cutoff=params["trap_sep"].magnitude/2,
               particle_color='#75b7ea',
               trap_color='gray',
               ax = ax)

plt.show()

## Testing for staircase

In [ ]:
def lmp2np(string):
    string = string.replace('sin','np.sin')
    string = string.replace('cos','np.cos')
    string = string.replace('PI','np.pi')
    string = string.replace('e6','')
    return string

In [ ]:
omega = np.pi/2/300

# time taken to rotate by one degree
t_deg = (5*ureg.deg).to(ureg.rad).magnitude / omega
# time for evolution
t_evo = 100 
N = np.round(np.pi/2/omega/t_deg)
total_time = N*(t_deg+t_evo)
time = np.arange(0,total_time,0.001)
print(f'Time for 1 deg: {t_deg:.2f}')
print(f'Number of slices: {N:.2f}')
print(f'Total time: {total_time:.2f}')

In [ ]:
current_time = 0
fieldx = []
for n in range(0,int(N)):
    fieldx.append(f'cos({omega}*(time-{current_time-n*t_deg}))*(time>{current_time})*(time<={current_time+t_deg})')
        
    current_time += t_deg
    fieldx.append(f'cos(omega*{(n+1)*t_deg})*(time>{current_time})*(time<={current_time+t_evo})')
    current_time += t_evo

In [ ]:
x = eval(lmp2np('+'.join(fieldx)))

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
ax.plot(time,x)
ax.grid()

## Trying to make animations

In [5]:
rootpath = '/home/frieren/BIG/reentrancy/test11/10/3/20mT/'
trj = ice.trajectory(os.path.join(rootpath,'xtrj1.csv'))
trj.load()
v = trj.get_vertices(os.path.join(rootpath,'vertices1.csv'))

In [7]:
if not os.path.isdir(savepath):
    os.mkdir(savepath)

trj.paint_frame(v,params,savepath,
                frame=50,
                set_title=f'$t = {50/20:.2f} s$',
                region=[0,300,0,300],
               )

In [4]:
savepath = os.path.join(rootpath,'frames')
frames = trj.trj.index.get_level_values('frame').unique().to_numpy()
time = frames/params['framespersec'].magnitude
titles = [f'$t = {t:.2f} s$' for t in time]

NameError: name 'datapath' is not defined

In [ ]:

ice.export_frames(trj,v,params,savepath,frames,titles,workers=5)